Import packages and define functions

In [14]:
import numpy as np
import datetime
import pandas as pd
import xarray as xa

from scipy.stats.stats import pearsonr
from scipy.stats import beta
from statsmodels.tsa import stattools
from netCDF4 import Dataset
from glob import glob

from tqdm import tqdm

In [2]:
def to_decimal_year(time):
    dt = pd.to_datetime(time) # converts np.datetime64 into datetime object
    dec_year = []
    for i in range(len(dt)):
        start = datetime.date(dt[i].year,1,1).toordinal()
        year_length = datetime.date(dt[i].year+1, 1, 1).toordinal() - start
        dec_year.append(dt[i].year + (float(dt[i].toordinal() - start) / year_length)) # calculates how much time has passed in the year, divides by year length, then adds to start year
    return np.array(dec_year)

In [3]:
def to_decimal_year(time):
    dt = pd.to_datetime(time) # converts np.datetime64 into datetime object
    dec_year = []
    for i in range(len(dt)):
        start = datetime.date(dt[i].year,1,1).toordinal()
        year_length = datetime.date(dt[i].year+1, 1, 1).toordinal() - start
        dec_year.append(dt[i].year + (float(dt[i].toordinal() - start) / year_length)) # calculates how much time has passed in the year, divides by year length, then adds to start year
    return np.array(dec_year)
def lag_corr(array1,array2,lag,T=None,return_autocorrelation_lengthscale = False,nlags=1000):
    if T is None:
        T = (2*np.max(np.cumsum(stattools.acf(array1,nlags=nlags)))+2*np.max(np.cumsum(stattools.acf(array2,nlags=nlags))))
    if lag>0:
        r = pearsonr(array2[lag:-1],array1[:-1-lag])[0]
    elif lag<0:
        l = int(-1.*lag)
        r = pearsonr(array1[l:-1],array2[:-1-l])[0]
    else:
        r = pearsonr(array1,array2)[0]
    n = (len(array1)-lag)*2/T - 14 # 1 for mean, 1 for trend, 6*2 for 6 seasonal harmonics
    dist = beta(n/2-1,n/2-1,loc=-1,scale=2)
    p = 2*dist.cdf(-abs(r))
    if return_autocorrelation_lengthscale:
        return r,p,T
    else:
        return r,p
def detrend_from_coefs(time,values,coefs):
    yr = to_decimal_year(time)
    
    mean = coefs[0]
    trend = coefs[1]*(yr-np.mean(yr))
    res_coefs = coefs[2:]
    
    detrended = values.copy()
    detrended -= mean
    detrended -= trend
    for j in range(0,len(res_coefs),2):
        omega = (j+2)*np.pi
        detrended -= (res_coefs[j]*np.sin(omega*yr))+(res_coefs[j+1]*np.cos(omega*yr))
    
    return detrended  

Extract data

In [4]:
files = np.sort(glob('/Users/erickson/Documents/Data/RFROM/*.nc'))
data = xa.open_mfdataset(files)
time = data['time'].values

In [5]:
coefs = Dataset('trend.nc','r')

In [6]:
dep = 150
depind = np.where(np.isin(data.variables['mean_pressure'].values,dep))[0][0]

In [7]:
lags = np.arange(-52*3,52*3+1)
lats = data['latitude'].values
lons = data['longitude'].values
lagged_corr = np.empty(shape=(len(lats),len(lons),len(lags)))*np.nan
lagged_p = np.empty(shape=(len(lats),len(lons),len(lags)))*np.nan
lagged_n = np.empty(shape=(len(lats),len(lons),len(lags)))*np.nan

In [44]:
nc_out = Dataset('lagged_correlation_analysis_150m_final.nc','w')
nc_out.createDimension('lag',len(lags))
nc_out.createDimension('lat',len(lats))
nc_out.createDimension('lon',len(lons))
var = nc_out.createVariable('lag','i8',('lag'))
var.units = 'weeks'
var[:] = lags
nc_out.createVariable('lat','f4',('lat'))[:] = lats
nc_out.createVariable('lon','f4',('lon'))[:] = lons
nc_out.createVariable('lagged_corr','f4',('lag','lat','lon'))
nc_out.createVariable('lagged_p','f4',('lag','lat','lon'))
nc_out.createVariable('lagged_n','f4',('lag','lat','lon'))
nc_out.close()

In [9]:
def lag_corr(array1,array2,lag,nlags=200):
    T1 = 2*np.max(np.cumsum(stattools.acf(array1,nlags=nlags)))
    T2 = 2*np.max(np.cumsum(stattools.acf(array2,nlags=nlags)))
    
    if lag>0:
        r = pearsonr(array2[lag:-1],array1[:-1-lag])[0]
    elif lag<0:
        l = int(-1.*lag)
        r = pearsonr(array1[l:-1],array2[:-1-l])[0]
    else:
        r = pearsonr(array1,array2)[0]
    l = len(array1)-np.abs(lag)
    n = (l/T1 + l/T2) - 8
    dist = beta(n/2-1,n/2-1,loc=-1,scale=2)
    p = 2*dist.cdf(-abs(r))
    return r,p,n

In [20]:
np.where(lats<-50)[0]

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39])

In [24]:
for j in tqdm(range(40)): #tqdm(range(len(lats))):
    for k in range(len(lons)):
        if np.isnan(coefs.variables['coefs'][0,0,j,k]):
            continue
        if np.any(np.isnan(coefs.variables['coefs'][:,0,j,k])) or np.any(np.isnan(coefs.variables['coefs'][:,depind,j,k])):
            continue
        surf_data = detrend_from_coefs(time,data.variables['ocean_temperature'][:,0,j,k].values,coefs.variables['coefs'][:,0,j,k])
        dep_data = detrend_from_coefs(time,data.variables['ocean_temperature'][:,depind,j,k].values,coefs.variables['coefs'][:,depind,j,k])
        if np.any(np.isnan(dep_data)) or np.any(np.isinf(dep_data)) or np.any(np.isnan(surf_data)) or np.any(np.isinf(surf_data)):
            continue
        for l in range(len(lags)):
            lag = lags[l]
            lagged_corr,lagged_p,lagged_n = lag_corr(surf_data,dep_data,lag);
            nc_out = Dataset('lagged_correlation_analysis_150m_final.nc','a')
            nc_out.variables['lagged_corr'][l,j,k] = lagged_corr
            nc_out.variables['lagged_p'][l,j,k] = lagged_p
            nc_out.variables['lagged_n'][l,j,k] = lagged_n
            nc_out.close()

100%|██████████████████████████████████████| 40/40 [7:36:15<00:00, 684.39s/it]
